In [1]:
import pandas as pd
import numpy as np
import yaml
import os
import sys

##### IO hadling 

In [3]:
with open('../excel_files/Spacemonk/data.yml') as file:
    yml_data = yaml.load(file, Loader=yaml.FullLoader)
file.close()

In [4]:
yml_data

{'data_directory': 'D:\\personal\\spacemonk\\excel_parser\\excel_files\\Spacemonk\\input',
 'workorder_directory': 'D:\\personal\\spacemonk\\excel_parser\\excel_files\\Spacemonk\\output',
 'upper matrix header': 'activity description',
 'coordinate term': 'frequency'}

In [5]:
os.chdir(yml_data['data_directory'])

'D:\\personal\\spacemonk\\excel_parser\\excel_files\\Spacemonk\\input'

In [8]:
each_file =0
for files in os.walk(yml_data['data_directory']):
    for each_file in files[2]:
        if ".xlsx" in each_file:
            print("Data File found:\t",each_file)
        
    if(each_file!=0):
        data=each_file
    else:
        print("Data not found")
        sys.exit(0)

Data File found:	 Summit 52 week PPM Schedule-2019.xlsx


##### Data Manipulation 

In [9]:
file = pd.ExcelFile(data) 

In [10]:
df = pd.read_excel(file,sheet_name=file.sheet_names[0],header = None)

In [11]:
# drops the complete null rows and columns in a df
def df_dropna(df):
    df.dropna(axis=0,how="all",inplace=True)
    df.dropna(axis=1,how="all",inplace=True)


In [12]:
df_dropna(df)

In [13]:
# Mapfunction to convert all the string in df to lowercase and keeping the other types same
def df_to_lower(df):
    return df.applymap(lambda s:s.lower() if type(s) == str else s)

In [14]:
df =df_to_lower(df)

In [15]:
# setting the index of df back from 0
def reset_index(df):
    df.reset_index(inplace=True)
    df.drop(df.columns[0], axis=1,inplace=True)

In [16]:
#setting the header, by the first row
def set_column(df):
    df.columns = df.iloc[0]
    df.drop(df.index[0],inplace=True)

In [17]:
# function to slice the header having the logo and other unuseful information
def upper_matrix_slice(df,header):
    x =df.iloc[:,0].str.find(header.lower()) # find() returns complete index of series, searches in first column of df
    x=int(x.index[x==0][0]) # find the true index of value and returns int 
    df =df.iloc[x:,:] # slicing the df before the header row
    return df

In [18]:
# function to get first occurance coordinates (row,column) of a string in a dataframe
def coordinates(df,term):
    for i in range(len(df.columns)):
        x=df[df.columns[i]].str.find(term)
        if(len(x.index[x==0])==0):
            pass
        else:
            x=int(x.index[x==0][0])
            return (x,i)  

In [19]:
def week_list(df):
  # finds the count of non nan
  for i in range(1,df.shape[1]+1):
        df.loc[df[i]==df[i],i]=i
  week_num = [[ j for j in i if j ==j] for i in np.array(df) ]
  return week_num

In [20]:
df =upper_matrix_slice(df,yml_data['upper matrix header'])

In [21]:
reset_index(df)

In [22]:
freq_c =coordinates(df,yml_data['coordinate term'])

In [23]:
df_freq = df.iloc[freq_c[0]:,freq_c[1]+1:]

In [24]:
df_act = df.iloc[0:,0:freq_c[1]+1]

In [25]:
set_column(df_act)

In [26]:
set_column(df_freq)

In [27]:
df_act["week number"] = week_list(df_freq)

In [28]:
df_act.loc[df_act.frequency!=df_act.frequency ,'week number'] = "---"
df_act.loc[df_act.frequency=='daily' ,'week number'] = "d"
df_act.loc[df_act.frequency=='weekly' ,'week number'] = "w"

##### EXPORTING DATA

In [30]:
df_act.to_excel(yml_data["workorder_directory"]+"\\"+file.sheet_names[0]+".xlsx")